In [47]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import datetime
import numpy as np

In [5]:
url_list = ['https://www.pgatour.com/stats/stat.112.2018.html', #par 3 birdie or better
       'https://www.pgatour.com/stats/stat.447.2018.html', #par 4 eagles
       'https://www.pgatour.com/stats/stat.113.2018.html', #par 4 birdies or better
       'https://www.pgatour.com/stats/stat.448.2018.html', #par 5 eagles
       'https://www.pgatour.com/stats/stat.114.2018.html', #par 5 birdies or better
       'https://www.pgatour.com/stats/stat.02419.2018.html', #bogey average
       'https://www.pgatour.com/stats/stat.02415.2018.html'] #birdie to bogey ratio (to get DB)

In [6]:
filenames = ['par3_bird_2018.csv', 'par4_eag_2018.csv', 'par4_bird_2018.csv',
             'par5_eag_2018.csv', 'par5_bird_2018.csv', 'bogey_2018.csv',
             'doublebogey_2018.csv']

In [18]:
def get_date(url_list, filenames):
    sleep = 30
    for idx, url in enumerate(url_list):
        if idx != 0:
            print(f"sleeping {sleep} seconds")
            time.sleep(sleep)
        print(f"fetching {url}")
        f = requests.get(url, verify=False).text #gets the html text
        html = BeautifulSoup(f.replace('\n',''), 'html.parser') #parses the html
        csv_lines = [] #initiate empty csv list
        table = html.find('table', class_='table-styled') #get the headers
        headings = [t.text for t in table.find('thead').find_all('th')]
        csv_lines.append(headings)
        for tr in table.find('tbody').find_all('tr'): #get all the data
            info = [td.text.replace(u'\xa0', u' ').strip() for td in tr.find_all('td')]
            csv_lines.append(info)
        cur_filename = filenames[idx]
        print(f"writing to {cur_filename}")
        with open(cur_filename, "w") as output: #save the data to file
            writer = csv.writer(output, delimiter=',', lineterminator='\n')
            writer.writerows(csv_lines)
        #Sleep a bit longer so the pattern isn't so obvious.
        #Don't want to get our IP blocked.
        sleep += 5

    print("finished fetching data")

In [13]:
# A list of url's with the relevant 2018 statistics
url_list = ['https://www.pgatour.com/stats/stat.112.html', #par 3 birdie or better
       'https://www.pgatour.com/stats/stat.447.html', #par 4 eagles
       'https://www.pgatour.com/stats/stat.113.html', #par 4 birdies or better
       'https://www.pgatour.com/stats/stat.448.html', #par 5 eagles
       'https://www.pgatour.com/stats/stat.114.html', #par 5 birdies or better
       'https://www.pgatour.com/stats/stat.02419.html', #bogey average
       'https://www.pgatour.com/stats/stat.02415.html'] #birdie to bogey ratio (to get DB)

In [15]:
filenames = ['par3_bird_2019.csv', 'par4_eag_2019.csv', 'par4_bird_2019.csv', 
             'par5_eag_2019.csv', 'par5_bird_2019.csv', 'bogey_2019.csv',
             'doublebogey_2019.csv']

In [16]:
get_date(url_list, filenames)

fetching https://www.pgatour.com/stats/stat.112.html
writing to par3_bird_2019.csv
sleeping 35 seconds
fetching https://www.pgatour.com/stats/stat.447.html
writing to par4_eag_2019.csv
sleeping 40 seconds
fetching https://www.pgatour.com/stats/stat.113.html
writing to par4_bird_2019.csv
sleeping 45 seconds
fetching https://www.pgatour.com/stats/stat.448.html
writing to par5_eag_2019.csv
sleeping 50 seconds
fetching https://www.pgatour.com/stats/stat.114.html
writing to par5_bird_2019.csv
sleeping 55 seconds
fetching https://www.pgatour.com/stats/stat.02419.html
writing to bogey_2019.csv
sleeping 60 seconds
fetching https://www.pgatour.com/stats/stat.02415.html
writing to doublebogey_2019.csv
sleeping 65 seconds


In [19]:
df_bog = pd.read_csv('bogey_2018.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'TOTAL BOGEYS', 'ROUNDS PLAYED'])
df_bog = df_bog.rename(index=str, columns={'TOTAL BOGEYS':'B', 'ROUNDS PLAYED':'Rounds'})

df_db = pd.read_csv('doublebogey_2018.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'TOTAL BOGEYS AND WORSE'])
df_db = df_db.rename(index=str, columns={'TOTAL BOGEYS AND WORSE':'DBoW'})

df_bir3 = pd.read_csv('par3_bird_2018.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'PAR 3 BIRDIES OR BETTER', 'PAR 3 HOLES'])
df_bir3 = df_bir3.rename(index=str, columns={'PAR 3 BIRDIES OR BETTER':'P3b', 'PAR 3 HOLES':'P3H'})

df_eag4 = pd.read_csv('par4_eag_2018.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'TOTAL']) 
df_eag4 = df_eag4.rename(index=str, columns={'TOTAL':'P4e'})

df_bir4 = pd.read_csv('par4_bird_2018.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'PAR 4 BIRDIES OR BETTER', 'PAR 4 HOLES'])
if df_bir4['PAR 4 HOLES'].dtype == 'O': #need this if the number includes commas and quotes
    df_bir4['PAR 4 HOLES'] = df_bir4['PAR 4 HOLES'].str.replace(',', '').astype(float)
df_bir4 = df_bir4.rename(index=str, columns={'PAR 4 BIRDIES OR BETTER':'P4be', 'PAR 4 HOLES':'P4H'})

df_eag5 = pd.read_csv('par5_eag_2018.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'TOTAL'])
df_eag5 = df_eag5.rename(index=str, columns={'TOTAL': 'P5e'})

df_bir5 = pd.read_csv('par5_bird_2018.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'PAR 5 BIRDIES OR BETTER', 'PAR 5 HOLES'])
df_bir5 = df_bir5.rename(index=str, columns={'PAR 5 BIRDIES OR BETTER':'P5be', 'PAR 5 HOLES':'P5H'})

In [22]:
#%% Joining to create overall dataframe
#Now I join them all
df = df_bog.join(df_db).join(df_bir3).join(df_eag4).join(df_bir4).join(df_eag5).join(df_bir5)
df = df.fillna(0) #this is just for eagle

In [23]:
#Now subtract eagles from birdies or better to get just birdies
df['P4b'] = df['P4be'].subtract(df['P4e'])
df = df.drop('P4be', axis=1)

In [24]:
df['P5b'] = df['P5be'].subtract(df['P5e'])
df = df.drop('P5be', axis=1)

In [25]:
#Now subtract bogeys from bogeys or worse to get double bogeys (or worse)
df['DB'] = df['DBoW'].subtract(df['B'])
df = df.drop('DBoW', axis=1)

In [26]:
#%% Saving dataframe
df.to_pickle('stats_2018.pkl')

In [27]:
df.head()
"""
B = Bogeys
R = Rounds
P3b = Par 3 birdies
P3H = Par 3 holes
P4e = Par 4 eagles
P4H = Par 4 holes
P5e = Par 5 eagles
P5H = Par 5 holes
P4b = Par 4 birdies
P5b = Par 5 birides
DB = Double bogies or worse
"""

,B,Rounds,P3b,P3H,P4e,P4H,P5e,P5H,P4b,P5b,DB
PLAYER NAME,,,,,,,,,,,
Henrik Stenson,110,56,19,222,0.0,618.0,5.0,168,108.0,77.0,13
Rickie Fowler,147,74,45,288,0.0,832.0,7.0,212,154.0,97.0,22
Webb Simpson,186,92,37,368,3.0,1036.0,15.0,252,191.0,102.0,20
Dustin Johnson,151,74,55,291,5.0,817.0,10.0,224,179.0,114.0,17
Brooks Koepka,131,62,36,240,0.0,712.0,8.0,164,141.0,83.0,28


In [38]:
now = datetime.datetime.now()
date = now.strftime('%Y-%m-%d')

In [39]:
df_bog = pd.read_csv('bogey_2019.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'TOTAL BOGEYS', 'ROUNDS PLAYED'])
df_bog = df_bog.rename(index=str, columns={'TOTAL BOGEYS':'B', 'ROUNDS PLAYED':'Rounds'})

df_db = pd.read_csv('doublebogey_2019.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'TOTAL BOGEYS AND WORSE'])
df_db = df_db.rename(index=str, columns={'TOTAL BOGEYS AND WORSE':'DBoW'})

df_bir3 = pd.read_csv('par3_bird_2019.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'PAR 3 BIRDIES OR BETTER', 'PAR 3 HOLES'])
df_bir3 = df_bir3.rename(index=str, columns={'PAR 3 BIRDIES OR BETTER':'P3b', 'PAR 3 HOLES':'P3H'})

df_eag4 = pd.read_csv('par4_eag_2019.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'TOTAL']) 
df_eag4 = df_eag4.rename(index=str, columns={'TOTAL':'P4e'})

df_bir4 = pd.read_csv('par4_bird_2019.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'PAR 4 BIRDIES OR BETTER', 'PAR 4 HOLES'])
if df_bir4['PAR 4 HOLES'].dtype == 'O': #need this if the number includes commas and quotes
    df_bir4['PAR 4 HOLES'] = df_bir4['PAR 4 HOLES'].str.replace(',', '').astype(float)
df_bir4 = df_bir4.rename(index=str, columns={'PAR 4 BIRDIES OR BETTER':'P4be', 'PAR 4 HOLES':'P4H'})

df_eag5 = pd.read_csv('par5_eag_2019.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'TOTAL'])
df_eag5 = df_eag5.rename(index=str, columns={'TOTAL': 'P5e'})

df_bir5 = pd.read_csv('par5_bird_2019.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'PAR 5 BIRDIES OR BETTER', 'PAR 5 HOLES'])
#df_bir5['%'] = df_bir5['%']/100
df_bir5 = df_bir5.rename(index=str, columns={'PAR 5 BIRDIES OR BETTER':'P5be', 'PAR 5 HOLES':'P5H'})

#%% Joining to create overall dataframe
#Now I join them all
df = df_bog.join(df_db).join(df_bir3).join(df_eag4).join(df_bir4).join(df_eag5).join(df_bir5)
df = df.fillna(0) #this is just for eagle

#Now subtract eagles from birdies or better to get just birdies
df['P4b'] = df['P4be'].subtract(df['P4e'])
df = df.drop('P4be', axis=1)

df['P5b'] = df['P5be'].subtract(df['P5e'])
df = df.drop('P5be', axis=1)

#Now subtract bogeys from bogeys or worse to get double bogeys (or worse)
df['DB'] = df['DBoW'].subtract(df['B'])
df = df.drop('DBoW', axis=1)

#%% Saving dataframe
df.to_pickle('stats_'+date+'.pkl')

In [42]:
df_2018 = pd.read_pickle("stats_2018.pkl")

In [43]:
df_2019 = pd.read_pickle(f"stats_{date}.pkl")

In [44]:
df_stats = df_2018.join(df_2019, how='outer', lsuffix='_2018', rsuffix='_2019')

In [45]:
df_stats = df_stats.fillna(0)

In [46]:
#need to assign weght to newest year
weight=2 #this just doubles the importance of the newest stats
df_stats['B'] = df_stats['B_2018'] + df_stats['B_2019']*weight
df_stats['DB'] = df_stats['DB_2018'] + df_stats['DB_2019']*weight
df_stats['Rounds'] = df_stats['Rounds_2018'] + df_stats['Rounds_2019']*weight
df_stats['P3b'] = df_stats['P3b_2018'] + df_stats['P3b_2019']*weight
df_stats['P3H'] = df_stats['P3H_2018'] + df_stats['P3H_2019']*weight
df_stats['P4e'] = df_stats['P4e_2018'] + df_stats['P4e_2019']*weight
df_stats['P4H'] = df_stats['P4H_2018'] + df_stats['P4H_2019']*weight
df_stats['P4b'] = df_stats['P4b_2018'] + df_stats['P4b_2019']*weight
df_stats['P5e'] = df_stats['P5e_2018'] + df_stats['P5e_2019']*weight
df_stats['P5H'] = df_stats['P5H_2018'] + df_stats['P5H_2019']*weight
df_stats['P5b'] = df_stats['P5b_2018'] + df_stats['P5b_2019']*weight
df_stats['Real_Rounds']=df_stats['Rounds_2018'] + df_stats['Rounds_2019']

In [48]:
par = 72
par3 = 4
par4 = 10
par5 = 4
if par3 + par4 + par5 != 18:
    raise NameError("Holes don't add up to 18. Something is off.")

In [49]:
#%% Set the draftkings scoring system
pts_b = 3
pts_e = 8
pts_p = 0.5
pts_B = -0.5
pts_DB = -1

pts_streak = 3
pts_BF = 3
pts_4U70 = 5

def pts_place_tmp(plc):
    if plc>50:
        return 0
    if plc>40:
        return 1
    if plc>30:
        return 2
    if plc>25:
        return 3
    if plc>20:
        return 4
    if plc>15:
        return 5
    if plc>10:
        return 6
    if plc==10:
        return 7
    if plc==9:
        return 8
    if plc==8:
        return 9
    if plc==7:
        return 10
    if plc==6:
        return 12
    if plc==5:
        return 14
    if plc==4:
        return 16
    if plc==3:
        return 18
    if plc==2:
        return 20
    if plc==1:
        return 30

pts_place = np.vectorize(pts_place_tmp) #This makes it element wise for np.arrays

In [170]:
# Need to enter draft kings logic here.
df_stats[df_stats['B']== 0]

,B_2018,Rounds_2018,P3b_2018,P3H_2018,P4e_2018,P4H_2018,P5e_2018,P5H_2018,P4b_2018,P5b_2018,...,P5H,P5b,Real_Rounds,B/R,DB/R,3b/H,4e/H,4b/H,5e/H,5b/H
PLAYER NAME,,,,,,,,,,,,,,,,,,,,,


In [50]:
from itertools import groupby

def len_iter(items):
    return sum(1 for _ in items)
    
def consecutive_one(data):
    return max(len_iter(run) for val, run in groupby(data) if val)

# #now time for the monte carlo analysis
trials=100000
results = np.empty(trials) 
final_results = np.zeros(19)   
for b in range(3,13):
    for k in range(trials):
        vec = np.append(np.zeros(18-b), np.ones(b))
        np.random.shuffle(vec)
        results[k] = consecutive_one(vec)
    final_results[b] = np.mean(results>=3)
final_results[13:]=1

In [51]:
final_results

array([0.     , 0.     , 0.     , 0.01991, 0.07403, 0.17166, 0.31579,
       0.48848, 0.67159, 0.82888, 0.93419, 0.98423, 0.99866, 1.     ,
       1.     , 1.     , 1.     , 1.     , 1.     ])

In [84]:
streak_prob = {
    0:0,
    1:0,
    2:0,
    3:0.01991, 
    4:0.07403, 
    5:0.17166, 
    6:0.31579,
    7:0.48848, 
    8:0.67159, 
    9:0.82888, 
    10:0.93419, 
    11:0.98423, 
    12:0.99866, 
    13:1,
    14:1, 
    15:1, 
    16:1,
    17:1, 
    18:1     
}


In [54]:
sims = 10000
#%% Calculate poisson rates
df_stats['B/R'] = df_stats['B'].div(df_stats['Rounds']) 
df_stats['DB/R'] = df_stats['DB'].div(df_stats['Rounds']) 
df_stats['3b/H'] = df_stats['P3b'].div(df_stats['P3H'])
df_stats['4e/H'] = df_stats['P4e'].div(df_stats['P4H'])
df_stats['4b/H'] = df_stats['P4b'].div(df_stats['P4H']) 
df_stats['5e/H'] = df_stats['P5e'].div(df_stats['P5H'])
df_stats['5b/H'] = df_stats['P5b'].div(df_stats['P5H'])

In [55]:
df_stats

,B_2018,Rounds_2018,P3b_2018,P3H_2018,P4e_2018,P4H_2018,P5e_2018,P5H_2018,P4b_2018,P5b_2018,...,P5H,P5b,Real_Rounds,B/R,DB/R,3b/H,4e/H,4b/H,5e/H,5b/H
PLAYER NAME,,,,,,,,,,,,,,,,,,,,,
Aaron Baddeley,205.0,78.0,25.0,317.0,1.0,843.0,8.0,244.0,142.0,93.0,...,628.0,247.0,133.0,2.420213,0.271277,0.104084,0.001502,0.175263,0.022293,0.393312
Aaron Wise,234.0,88.0,47.0,357.0,1.0,959.0,5.0,268.0,196.0,119.0,...,616.0,289.0,141.0,2.649485,0.355670,0.153946,0.000476,0.197813,0.014610,0.469156
Abraham Ancer,231.0,96.0,49.0,386.0,0.0,1044.0,5.0,298.0,174.0,126.0,...,754.0,316.0,168.0,2.437500,0.275000,0.134096,0.000768,0.183564,0.019894,0.419098
Adam Hadwin,230.0,89.0,46.0,352.0,1.0,966.0,5.0,284.0,171.0,103.0,...,700.0,303.0,153.0,2.663594,0.276498,0.137931,0.001284,0.172517,0.027143,0.432857
Adam Long,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,360.0,116.0,56.0,2.625000,0.410714,0.102222,0.003317,0.154229,0.038889,0.322222
Adam Schenk,217.0,80.0,39.0,325.0,2.0,861.0,5.0,254.0,123.0,101.0,...,786.0,325.0,160.0,2.495833,0.325000,0.136972,0.001561,0.158018,0.026718,0.413486
Adam Scott,208.0,78.0,56.0,313.0,1.0,850.0,5.0,241.0,136.0,101.0,...,537.0,263.0,121.0,2.597561,0.280488,0.154780,0.002847,0.168565,0.027933,0.489758
Adam Svensson,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,340.0,136.0,51.0,2.823529,0.274510,0.144928,0.001848,0.153420,0.035294,0.400000
Alex Cejka,170.0,69.0,43.0,281.0,4.0,746.0,5.0,215.0,108.0,85.0,...,487.0,177.0,109.0,2.778523,0.248322,0.162029,0.002525,0.133838,0.014374,0.363450


In [56]:
num_players, tmp = df_stats.shape
birdies = np.empty([num_players, sims, 4])
for player_idx in range(num_players):
    cur_birdies3 = np.random.poisson(df_stats.iloc[player_idx]['3b/H']*par3, [1, sims, 4])
    cur_birdies4 = np.random.poisson(df_stats.iloc[player_idx]['4b/H']*par4, [1, sims, 4])
    cur_birdies5 = np.random.poisson(df_stats.iloc[player_idx]['5b/H']*par5, [1, sims, 4])
    cur_birdies = cur_birdies3+cur_birdies4+cur_birdies5
    birdies[player_idx,:,:] = cur_birdies
    
eagles = np.empty([num_players, sims, 4])
for player_idx in range(num_players):
    cur_eagles4 = np.random.poisson(df_stats.iloc[player_idx]['4e/H']*par4, [1, sims, 4])
    cur_eagles5 = np.random.poisson(df_stats.iloc[player_idx]['5e/H']*par5, [1, sims, 4])
    cur_eagles = cur_eagles4+cur_eagles5
    eagles[player_idx,:,:] = cur_eagles

bogeys = np.empty([num_players, sims, 4])
for player_idx in range(num_players):
    cur_bogeys = np.random.poisson(df_stats.iloc[player_idx]['B/R'], [1, sims, 4])
    bogeys[player_idx,:,:] = cur_bogeys
    
doublebogeys = np.empty([num_players, sims, 4])
for player_idx in range(num_players):
    cur_doublebogeys = np.random.poisson(df_stats.iloc[player_idx]['DB/R'], [1, sims, 4])
    doublebogeys[player_idx,:,:] = cur_doublebogeys
    
pars = np.full([num_players, sims, 4], 18)
pars = pars-birdies-eagles-bogeys-doublebogeys
pars[pars<0]=0

#%% Get the scores for each round
scores = np.full([num_players, sims, 4], par)
scores = scores - birdies - 2*eagles + bogeys +2*doublebogeys

In [131]:
# Get Player Simulated Rounds:
get_rounds = False
if get_rounds:
    m, n, r = scores.shape
    out_arr =  np.column_stack((np.repeat(np.arange(m),n), scores.reshape(m*n, -1)))
    out_df = pd.DataFrame(out_arr, columns=["Index", "round1", "round2", "round3", "round4"])
    print(out_df.head())
    df_stats.head()
    df_converted = df_stats.reset_index()
    merged = pd.merge(df_converted, out_df, left_index=True, right_on="Index")
    print(merged.columns)
    merged.head()
    merged[merged["PLAYER NAME"]=='Dustin Johnson'].describe()

In [61]:
birdies[0, 0:5]

array([[6., 3., 4., 6.],
       [6., 4., 6., 0.],
       [7., 6., 5., 4.],
       [2., 7., 5., 2.],
       [3., 2., 4., 9.]])

In [64]:
np.min(scores)

53.0

In [65]:
np.max(scores)

90.0

In [67]:
np.median(scores)

71.0

In [68]:
from scipy.stats import rankdata
#To finalize the scores array, 
#we have to go back and make the cut so that 
#only the top 70 golfers move on to rounds 3 and 4. 
#We use SciPy for this task. Keep in mind that it's 
#the top 70 golfers + ties that move on (meaning it 
#could be more than 70 that actually move on).

made_cut=np.empty([num_players, sims])
for cur_sim in range(sims):
    after2 = np.sum(scores[:,cur_sim,0:2], axis=1)
    made_cut[:,cur_sim] = rankdata(after2, method='min')<=70
missed_cut = (made_cut - 1) * (-1)

In [69]:
made_cut

array([[1., 1., 1., ..., 0., 0., 1.],
       [0., 1., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 0., 1., ..., 0., 1., 1.],
       [1., 1., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [71]:
num_players

240

In [72]:
dk_scores = np.zeros([num_players, sims, 4])

In [76]:
dk_scores = pts_b*birdies + pts_e*eagles + pts_B*bogeys + pts_DB*doublebogeys + pts_p*pars

In [96]:
scores

array([[[67., 70., 69., 65.],
        [68., 69., 68., 73.],
        [66., 72., 69., 70.],
        ...,
        [72., 73., 71., 69.],
        [71., 74., 72., 74.],
        [71., 69., 69., 71.]],

       [[78., 68., 68., 73.],
        [70., 67., 71., 68.],
        [69., 67., 71., 71.],
        ...,
        [64., 69., 73., 78.],
        [68., 72., 70., 72.],
        [68., 70., 73., 72.]],

       [[74., 66., 69., 71.],
        [71., 72., 66., 72.],
        [70., 67., 71., 76.],
        ...,
        [71., 72., 73., 71.],
        [74., 70., 77., 72.],
        [67., 71., 64., 73.]],

       ...,

       [[72., 70., 73., 69.],
        [71., 72., 72., 69.],
        [67., 68., 71., 67.],
        ...,
        [76., 71., 73., 71.],
        [65., 65., 75., 69.],
        [69., 71., 70., 73.]],

       [[69., 70., 66., 73.],
        [69., 71., 70., 71.],
        [71., 73., 69., 70.],
        ...,
        [68., 72., 70., 70.],
        [72., 71., 69., 76.],
        [73., 72., 75., 69.]],

       [[75.

In [86]:
#now lets add the streak bonus
def streak_bonus_tmp(be):
    prob=streak_prob[be]
    return np.random.binomial(1,prob)
streak_bonus = np.vectorize(streak_bonus_tmp )
bir_eag = birdies+eagles
bir_eag[bir_eag>18] = 18 #need to cap it at 18
streak_pts = streak_bonus(bir_eag)
dk_scores = dk_scores + streak_pts*pts_streak #adds the streak to each day that has it

#need to add bogey free bonus
bogeyFree = 1*(bogeys==0)
doublebogeyFree = 1*(doublebogeys==0)
allBFree = bogeyFree*doublebogeyFree
dk_scores = dk_scores + pts_BF*bogeyFree

#now lets add the placement bonus
total_score = np.sum(scores, axis=2)
#need to make sure we dont give placement bonuses to those missing the cut so we add 1000 strokes
total_score = total_score + missed_cut*1000
ranking = np.empty([num_players, sims])
for cur_sim in range(sims):
    ranking[:,cur_sim] = rankdata(total_score[:,cur_sim], method='min')
dk_placement_score = pts_place(ranking)
dk_scores[:,:,0] = dk_scores[:,:,0] + dk_placement_score #might as well add it onto the first day

#Now we need to add a bonus score if players have all 4 rounds below 70
A4U70_array=np.max(scores, axis=2)<70 
dk_scores[:,:,2] = dk_scores[:,:,2] + A4U70_array*pts_4U70 #we put it in this on the 3rd round so that it gets zeroed out if CUT

#Now we need to 0 out the scores from days 3 and 4 for those that did not make the cut
dk_scores[:,:,2] = dk_scores[:,:,2]*made_cut
dk_scores[:,:,3] = dk_scores[:,:,3]*made_cut

#Finally, we compress the scores
dk_final_score = np.sum(dk_scores, axis=2)

In [89]:
#%% Now let's calculate the statistics of importance
dk_median = np.median(dk_final_score, axis=1)
dk_mean = np.mean(dk_final_score, axis=1)
dk_max = np.max(dk_final_score, axis=1)
dk_min = np.min(dk_final_score, axis=1)
dk_25 = np.percentile(dk_final_score, 25, axis=1)
dk_75 = np.percentile(dk_final_score, 75, axis=1)
dk_makecut = np.mean(made_cut, axis=1)

#%% Now let's create the dataframe that we will pass to the linear program
df_results = pd.DataFrame(index=df_stats.index.copy())
#df_results['Salary']=df_dk['Salary']
df_results['dk_max']=dk_max
df_results['dk_min']=dk_min
df_results['dk_median']=dk_median
df_results['dk_mean']=dk_mean
df_results['dk_75']=dk_75
df_results['dk_25']=dk_25
df_results['dk_makecut']=dk_makecut
df_results['Rounds']=df_stats['Real_Rounds']
#df_results['Have_Data']=df_stats['Real']

In [163]:
df_results.sort_values(by='dk_mean', ascending=False).to_excel("draft_kings_3m_open.xlsx", index=False)